In [ ]:
#claude
#bu kod açıklamayı hızlı hızlı iyi çeken kod ama öbür bilgileri de çekmek istiyorum

import os
import time
import random
import pandas as pd
import signal
import sys
import ctypes
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from markdownify import markdownify as md

# === [0] Uyku Modunu Engelle ===
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001
ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)

# === [1] Konfigürasyon ===
CHROME_DRIVER_PATH = r"C:\Users\emirhan.gul\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
INPUT_FILE = "yeni_ilan_linkleri.csv"
OUTPUT_FILE = "ilan_aciklamalari.csv"
SAVE_INTERVAL = 20  # Her 20 ilanda bir kaydet
DRIVER_RESTART_INTERVAL = 150  # Her 150 ilanda bir driver'ı yeniden başlat
MAX_RETRIES = 3  # Maksimum deneme sayısı

# === [2] Global Değişkenler ===
driver = None
results_list = []
processed_urls = set()

# === [3] Chrome Driver Ayarları ===
def get_chrome_options():
    """Chrome seçeneklerini ayarla"""
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    return options

def init_driver():
    """Yeni Chrome driver başlat"""
    global driver
    if driver:
        try:
            driver.quit()
        except:
            pass
    
    service = Service(CHROME_DRIVER_PATH)
    options = get_chrome_options()
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_page_load_timeout(30)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    })
    print("✅ Yeni WebDriver oturumu başlatıldı.")
    return driver

# === [4] Sayfa Navigasyonu ===
def navigate_to_page(driver, url, timeout=3):
    """Sayfaya git, timeout olursa stop() uygula ve scroll yap"""
    try:
        driver.set_page_load_timeout(timeout)
        driver.get(url)
        human_like_scroll(driver)  # 👈 küçük scroll ve kısa bekleme
        return True
    except TimeoutException:
        print(f"⚠️ Sayfa zaman aşımı, stop() uygulandı: {url}")
        try:
            driver.execute_script("window.stop();")
            human_like_scroll(driver)  # 👈 scroll yine yapılmalı
            return True
        except Exception as e:
            print(f"❌ window.stop() çalıştırılamadı: {e}")
            return False
    except Exception as e:
        print(f"❌ Sayfa yüklenirken hata: {e}")
        return False



def human_like_scroll(driver):
    try:
        driver.execute_script("window.scrollTo(0, 200);")  # sadece biraz aşağı kaydır
        time.sleep(random.uniform(0.5, 1.0))  # kısa ama insan gibi bir bekleme
    except:
        pass


# === [5] Açıklama Çekme ===
def get_description(driver, url):
    """İlan açıklamasını çek"""
    
    for attempt in range(MAX_RETRIES):
        print(f"  🔄 Deneme {attempt + 1}/{MAX_RETRIES}: {url}")
        
        # Sayfaya git
        if not navigate_to_page(driver, url, timeout=3):
            if attempt < MAX_RETRIES - 1:
                print("   ⏳ Tekrar deneniyor...")
                time.sleep(random.uniform(1, 3))
                continue
            else:
                print(f"   ❌ Maksimum deneme aşıldı: {url}")
                return ""
        
        try:
            # Açıklama elementini bekle
            parent = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "tab-description"))
            )
            content_div = parent.find_element(By.TAG_NAME, "div")
            inner_html = content_div.get_attribute("innerHTML")
            
            # Markdown'a çevir ve temizle
            description_md = md(inner_html, heading_style="ATX")
            clean_description = description_md.strip()
            
            print(f"   ✅ Açıklama alındı ({len(clean_description)} karakter)")
            return clean_description
                      
        except (NoSuchElementException, TimeoutException) as e:
            print(f"   ⚠️ Element bulunamadı: {type(e).__name__}")
            print(f"   ❌ Açıklama çekilemedi: {url}")
            return ""
        except Exception as e:
            print(f"   ❌ Beklenmeyen hata: {e}")
            if attempt < MAX_RETRIES - 1:
                continue
            else:
                return ""
    
    return ""

# === [6] Kayıt Fonksiyonları ===
def save_progress():
    """İlerlemeyi kaydet"""
    try:
        df = pd.DataFrame(results_list)
        df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
        print(f"💾 İlerleme kaydedildi: {len(results_list)} kayıt")
    except Exception as e:
        print(f"❌ Kayıt hatası: {e}")

def load_existing_data():
    """Mevcut veriyi yükle"""
    global results_list, processed_urls
    
    if os.path.exists(OUTPUT_FILE):
        try:
            df = pd.read_csv(OUTPUT_FILE, encoding="utf-8-sig")
            results_list = df.to_dict('records')
            processed_urls = set(df["ilan_linki"].dropna().tolist())
            print(f"📂 Mevcut {len(processed_urls)} kayıt yüklendi")
        except Exception as e:
            print(f"⚠️ Mevcut dosya yüklenemedi: {e}")
            results_list = []
            processed_urls = set()
    else:
        results_list = []
        processed_urls = set()

# === [7] Çıkış Fonksiyonu ===
def cleanup_and_exit():
    """Temizlik yap ve çık"""
    print("\n🛑 Program sonlandırılıyor...")
    save_progress()
    if driver:
        try:
            driver.quit()
        except:
            pass
    ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    print("✅ Temizlik tamamlandı. Çıkılıyor...")
    sys.exit(0)

# === [8] Signal Handler ===
def signal_handler(signum, frame):
    """Ctrl+C ile durdurulduğunda"""
    print("\n⚠️ Kullanıcı tarafından durduruldu (Ctrl+C)")
    cleanup_and_exit()

# === [9] Ana Fonksiyon ===
def main():
    global driver, results_list, processed_urls
    
    # Signal handler'ı ayarla
    signal.signal(signal.SIGINT, signal_handler)
    
    print("🚀 Araç İlanı Scraper Başlatılıyor...")
    
    # Mevcut veriyi yükle
    load_existing_data()
    
    # Input dosyasını oku
    try:
        df_links = pd.read_csv(INPUT_FILE, encoding="utf-8")
        all_urls = df_links["ilan_linki"].tolist()
    except Exception as e:
        print(f"❌ Input dosyası okunamadı: {e}")
        return
    
    # İşlenecek URL'leri filtrele
    urls_to_process = []
    for url in all_urls:
        if not url.startswith("http"):
            url = "https://www.arabam.com" + url
        if url not in processed_urls:
            urls_to_process.append(url)
    
    if not urls_to_process:
        print("✅ Tüm ilanlar zaten işlenmiş!")
        return
    
    print(f"📊 Toplam {len(all_urls)} ilan, {len(urls_to_process)} tanesi işlenecek")
    
    # Ana işlem döngüsü
    try:
        driver = init_driver()
        processed_this_session = 0
        
        for idx, url in enumerate(urls_to_process):
            current_index = len(processed_urls) + idx + 1
            print(f"\n[{current_index}/{len(all_urls)}] İşleniyor: {url}")
            
            # Belirli aralıklarla driver'ı yeniden başlat
            if processed_this_session > 0 and processed_this_session % DRIVER_RESTART_INTERVAL == 0:
                print(f"\n🔄 {DRIVER_RESTART_INTERVAL} ilan işlendi, WebDriver yeniden başlatılıyor...")
                driver = init_driver()
                time.sleep(random.uniform(3, 7))
            
            # Açıklamayı çek
            description = get_description(driver, url)
            results_list.append({"ilan_linki": url, "aciklama": description})
            processed_urls.add(url)
            processed_this_session += 1
            
            # Düzenli kayıt
            if (idx + 1) % SAVE_INTERVAL == 0 or (idx + 1) == len(urls_to_process):
                save_progress()
                time.sleep(random.uniform(1, 3))
            
            # İlanlar arası bekleme
            time.sleep(random.uniform(2, 4))
    
    except KeyboardInterrupt:
        print("\n⚠️ Program kullanıcı tarafından durduruldu")
        cleanup_and_exit()
    except Exception as e:
        print(f"\n🚨 Kritik hata: {e}")
        cleanup_and_exit()
    finally:
        cleanup_and_exit()

# === [10] Program Başlatma ===
if __name__ == "__main__":
    main()

🚀 Araç İlanı Scraper Başlatılıyor...
📂 Mevcut 465 kayıt yüklendi
📊 Toplam 2105 ilan, 1640 tanesi işlenecek
✅ Yeni WebDriver oturumu başlatıldı.

[466/2105] İşleniyor: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-comfortline/2016-mdl-passat-dolular-dolusu/29894708
  🔄 Deneme 1/3: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-comfortline/2016-mdl-passat-dolular-dolusu/29894708
⚠️ Sayfa zaman aşımı, stop() uygulandı: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-comfortline/2016-mdl-passat-dolular-dolusu/29894708
   ✅ Açıklama alındı (387 karakter)

[468/2105] İşleniyor: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-highline/soyoglu-dan-2015-vw-highline-cam-tavan-otonom-360-kam-71-000-km/29990564
  🔄 Deneme 1/3: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-highline/soyoglu-dan-2015-vw-highline-cam-tavan-o

2025-07-16 14:39:55,902 - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000213BB42E900>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')': /session/331301923045d3e7039be7efb34f6db5


In [ ]:
import os
import time
import random
import pandas as pd
import signal
import sys
import ctypes
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException, ElementClickInterceptedException
from markdownify import markdownify as md

# === [0] Uyku Modunu Engelle ===
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001
ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)

# === [1] Konfigürasyon ===
CHROME_DRIVER_PATH = r"C:\Users\emirhan.gul\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
INPUT_FILE = "yeni_ilan_linkleri.csv"
OUTPUT_FILE = "ilan_tum_bilgiler.csv"
SAVE_INTERVAL = 20  # Her 20 ilanda bir kaydet
DRIVER_RESTART_INTERVAL = 150  # Her 150 ilanda bir driver'ı yeniden başlat

# Sayfa yüklemesi ve element bekleme zaman aşımı ayarları
GLOBAL_PAGE_LOAD_TIMEOUT_SECONDS = 4
ELEMENT_WAIT_TIMEOUT_SECONDS = 7

# === [2] Global Değişkenler ===
driver = None
results_list = []
processed_urls = set()

# === [3] Chrome Driver Ayarları ===
def get_chrome_options():
    """Chrome seçeneklerini ayarla"""
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    return options

def init_driver():
    """Yeni Chrome driver başlat"""
    global driver
    if driver:
        try:
            driver.quit()
        except:
            pass
    
    service = Service(CHROME_DRIVER_PATH)
    options = get_chrome_options()
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_page_load_timeout(GLOBAL_PAGE_LOAD_TIMEOUT_SECONDS)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    })
    print(f"✅ Yeni WebDriver oturumu başlatıldı. Global sayfa yükleme zaman aşımı: {GLOBAL_PAGE_LOAD_TIMEOUT_SECONDS}s")
    return driver

# === [4] Yardımcı Fonksiyonlar ===

def human_like_scroll():
    """Sayfayı insan gibi rastgele kaydırır."""
    global driver
    try:
        scroll_amount = random.randint(200, 600)
        driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
        time.sleep(random.uniform(0.5, 1.0))
    except Exception as e:
        print(f"⚠️ Scroll hatası: {e}")
        pass

def extract_text_safe(by, selector):
    """Belirtilen selector'a göre metni güvenli bir şekilde çeker."""
    global driver
    try:
        return driver.find_element(by, selector).text.strip()
    except NoSuchElementException:
        return None
    except Exception as e:
        print(f"⚠️ extract_text_safe hatası ({selector}): {e}")
        return None

def get_property_value(key):
    """Detaylı özellikler listesinden belirli bir anahtarın değerini çeker."""
    global driver
    try:
        rows = driver.find_elements(By.CSS_SELECTOR, ".property-item")
        for row in rows:
            try:
                if key in row.text:
                    value_element = row.find_element(By.CLASS_NAME, "property-value")
                    return value_element.text.strip()
            except NoSuchElementException:
                continue
            except Exception as e:
                print(f"⚠️ get_property_value iç hata ({key}): {e}")
                continue
        return None
    except Exception as e:
        print(f"⚠️ get_property_value genel hata: {e}")
        return None

def extract_parca_boya():
    """Aracın parça, boya, değişen bilgilerini çeker."""
    global driver
    categories = {
        "Orjinal": [],
        "Lokal boyalı": [],
        "Boyalı": [],
        "Değişmiş": [],
        "Belirtilmemiş": []
    }

    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "car-damage-info")))
        section = driver.find_element(By.CLASS_NAME, "car-damage-info")
        items = section.find_elements(By.CLASS_NAME, "car-damage-info-item")

        for item in items:
            try:
                label = item.find_element(By.TAG_NAME, "p").text.strip()
                ul = item.find_element(By.TAG_NAME, "ul")
                for li in ul.find_elements(By.TAG_NAME, "li"):
                    text = li.text.strip()
                    if text:
                        categories.get(label, []).append(text)
            except NoSuchElementException:
                continue
            except Exception as e:
                print(f"⚠️ Boya bilgisi item işlenirken hata: {e}")
                continue

    except TimeoutException:
        print("⚠️ Boya/değişen bilgisi bölümü zaman aşımına uğradı veya bulunamadı.")
    except NoSuchElementException:
        print("⚠️ Boya/değişen bilgisi bölümü bulunamadı.")
    except Exception as e:
        print(f"❌ Boya bilgisi alınırken genel hata: {e}")
    
    flat_boya_data = {}
    for k, v in categories.items():
        if v:
            if k in ["Orjinal", "Lokal boyalı", "Boyalı", "Değişmiş", "Belirtilmemiş"]:
                flat_boya_data[k.lower().replace(" ", "_") + "_parcalar"] = ", ".join(v)
            else:
                flat_boya_data[k.lower().replace(" ", "_")] = ", ".join(v)
        else:
            if k in ["Orjinal", "Lokal boyalı", "Boyalı", "Değişmiş", "Belirtilmemiş"]:
                flat_boya_data[k.lower().replace(" ", "_") + "_parcalar"] = None
            else:
                flat_boya_data[k.lower().replace(" ", "_")] = None

    return flat_boya_data

# === [5] Sayfa Navigasyonu ===
def navigate_to_page(url):
    """
    Sayfaya git, belirlenen süre içinde yüklenmezse durdur ve gerekli elementlerin varlığını kontrol et.
    """
    global driver
    try:
        driver.get(url)
        
    except TimeoutException:
        print(f"⚠️ Sayfa yüklemesi {GLOBAL_PAGE_LOAD_TIMEOUT_SECONDS}s içinde tamamlanamadı, yükleme durduruluyor: {url}")
        try:
            driver.execute_script("window.stop()")
        except WebDriverException as e:
            print(f"❌ window.stop() çalıştırılamadı: {e}")
            return False
    except WebDriverException as e:
        print(f"❌ WebDriver hatası sayfa alırken ({type(e).__name__}): {e}")
        return False
    except Exception as e:
        print(f"❌ Navigasyon sırasında beklenmeyen hata ({type(e).__name__}): {e}")
        return False

    try:
        # Çerez kabul butonunu güvenli bir şekilde tıklamayı dene
        try:
            accept_btn = WebDriverWait(driver, 2).until( # Daha kısa bekleme süresi
                EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
            )
            accept_btn.click()
            print("🍪 Çerezler kabul edildi.")
            time.sleep(0.5) # Küçük bir bekleme ekleyelim
        except (TimeoutException, NoSuchElementException, ElementClickInterceptedException):
            # Buton bulunamazsa veya tıklanamazsa, devam et (zaten cookie kabul edilmiş olabilir)
            pass
        
        # Ana içeriğin yüklendiğini kontrol et
        WebDriverWait(driver, ELEMENT_WAIT_TIMEOUT_SECONDS).until(
            EC.presence_of_element_located((By.ID, "tab-description"))
        )
        print(f"✔️ Gerekli elementler (açıklama alanı) algılandı: {url}")
        human_like_scroll()
        return True
        
    except TimeoutException:
        print(f"⚠️ Temel elementler (örn. açıklama) {ELEMENT_WAIT_TIMEOUT_SECONDS}s içinde gelmedi: {url}")
        return False
    except NoSuchElementException:
        print(f"⚠️ Temel elementler sayfada bulunamadı: {url}")
        return False
    except Exception as e:
        print(f"❌ Element kontrolü sırasında beklenmeyen hata ({type(e).__name__}): {e}")
        return False

# === [6] Tüm İlan Verilerini Çekme Fonksiyonu ===
def get_all_ad_data(url):
    """
    İlanın açıklamasını ve diğer tüm detaylarını çekmeye çalışır.
    Bu fonksiyonda artık tekrar deneme (retry) mekanizması yoktur.
    Hata durumunda driver'ı yeniden başlatmak yerine sonraki ilana geçer.
    """
    global driver
    ad_data = {"ilan_linki": url} # Başlangıçta sadece linki ekle

    # Sayfaya git ve temel elementlerin yüklenmesini bekle (Tek Deneme)
    if not navigate_to_page(url):
        print(f"   ❌ İlan için sayfa yüklenemedi veya temel elementler bulunamadı. Sonraki ilana geçiliyor: {url}")
        # Eksik veri içeren bir sözlük döndür, ana döngü bunu işlenmemiş sayar.
        ad_data.update({
            "aciklama": None, "fiyat": None, "ilan_no": None, "ilan_tarihi": None,
            "marka": None, "seri": None, "model": None, "yıl": None, "km": None,
            "vites": None, "yakıt": None, "kasa": None, "renk": None,
            "motor_hacmi": None, "motor_gücü": None, "çekiş": None,
            "boya_durumu": None, "takasa_uygunluk": None, "kimden": None,
            "orjinal_parcalar": None, "lokal_boyali_parcalar": None,
            "boyali_parcalar": None, "değişmiş_parcalar": None, "belirtilmemiş_parcalar": None
        })
        return ad_data # Boş veya eksik veriyle dön

    # Eğer buraya geldiysek, sayfa navigasyonu ve temel elementler başarılı demektir.
    # Şimdi detayları çekebiliriz.

    # --- Açıklama Çekme ---
    try:
        parent = driver.find_element(By.ID, "tab-description")
        content_div = parent.find_element(By.TAG_NAME, "div")
        inner_html = content_div.get_attribute("innerHTML")
        ad_data["aciklama"] = md(inner_html, heading_style="ATX").strip()
        print(f"   ✅ Açıklama alındı ({len(ad_data['aciklama'])} karakter)")
    except (NoSuchElementException, TimeoutException):
        print("   ⚠️ Açıklama elementi bulunamadı.")
        ad_data["aciklama"] = None
    except Exception as e:
        print(f"   ❌ Açıklama çekerken beklenmeyen hata: {e}")
        ad_data["aciklama"] = None

    # --- Diğer İlan Bilgilerini Çekme ---
    ad_data["fiyat"] = extract_text_safe(By.CSS_SELECTOR, ".product-price")
    if ad_data["fiyat"]:
        ad_data["fiyat"] = ad_data["fiyat"].replace("TL", "").replace(".", "").strip()
        
    ad_data["ilan_no"] = get_property_value("İlan No")
    ad_data["ilan_tarihi"] = get_property_value("İlan Tarihi")
    ad_data["marka"] = get_property_value("Marka")
    ad_data["seri"] = get_property_value("Seri")
    ad_data["model"] = get_property_value("Model")
    ad_data["yıl"] = get_property_value("Yıl")
    ad_data["km"] = get_property_value("Kilometre")
    ad_data["vites"] = get_property_value("Vites Tipi")
    ad_data["yakıt"] = get_property_value("Yakıt Tipi")
    ad_data["kasa"] = get_property_value("Kasa Tipi")
    ad_data["renk"] = get_property_value("Renk")
    ad_data["motor_hacmi"] = get_property_value("Motor Hacmi")
    ad_data["motor_gücü"] = get_property_value("Motor Gücü")
    ad_data["çekiş"] = get_property_value("Çekiş")
    ad_data["boya_durumu"] = get_property_value("Durumu")
    ad_data["takasa_uygunluk"] = get_property_value("Takas")
    ad_data["kimden"] = get_property_value("Kimden")

    boya_data = extract_parca_boya()
    ad_data.update(boya_data)

    print("   ✅ Tüm ilan bilgileri çekildi.")
    return ad_data

# === [7] Kayıt Fonksiyonları ===
def save_progress():
    """İlerlemeyi kaydet"""
    try:
        df = pd.DataFrame(results_list)
        if os.path.exists(OUTPUT_FILE):
            df_old = pd.read_csv(OUTPUT_FILE, encoding="utf-8-sig")
            df_combined = pd.concat([df_old, df], ignore_index=True)
            df_combined.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
        else:
            df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
        
        print(f"💾 İlerleme kaydedildi: {len(results_list)} yeni kayıt eklendi.")
        results_list.clear()
    except Exception as e:
        print(f"❌ Kayıt hatası: {e}")

def load_existing_data():
    """Mevcut veriyi yükle ve işlenmiş URL'leri hazırla"""
    global results_list, processed_urls
    
    if os.path.exists(OUTPUT_FILE):
        try:
            df = pd.read_csv(OUTPUT_FILE, encoding="utf-8-sig")
            # Sadece 'ilan_linki' dolu olanları processed_urls'e ekle
            processed_urls = set(df["ilan_linki"].dropna().tolist())
            print(f"📂 Mevcut {len(processed_urls)} kayıt yüklendi (işlenmiş URL'ler).")
            results_list = []
        except Exception as e:
            print(f"⚠️ Mevcut dosya yüklenemedi: {e}. Sıfırdan başlıyor.")
            results_list = []
            processed_urls = set()
    else:
        results_list = []
        processed_urls = set()

# === [8] Çıkış Fonksiyonu ===
def cleanup_and_exit():
    """Temizlik yap ve çık"""
    print("\n🛑 Program sonlandırılıyor...")
    save_progress()
    if driver:
        try:
            driver.quit()
        except:
            pass
    ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    print("✅ Temizlik tamamlandı. Çıkılıyor...")
    sys.exit(0)

# === [9] Signal Handler ===
def signal_handler(signum, frame):
    """Ctrl+C ile durdurulduğunda"""
    print("\n⚠️ Kullanıcı tarafından durduruldu (Ctrl+C)")
    cleanup_and_exit()

# === [10] Ana Fonksiyon ===
def main():
    global driver, results_list, processed_urls
    
    signal.signal(signal.SIGINT, signal_handler)
    
    print("🚀 Araç İlanı Scraper Başlatılıyor...")
    
    load_existing_data()
    
    try:
        df_links = pd.read_csv(INPUT_FILE, encoding="utf-8")
        all_urls_raw = df_links["ilan_linki"].tolist()
        all_urls = []
        for url in all_urls_raw:
            if isinstance(url, str):
                if not url.startswith("http"):
                    url = "https://www.arabam.com" + url
                all_urls.append(url)
            else:
                print(f"⚠️ Geçersiz URL tipi atlandı: {url}")

    except Exception as e:
        print(f"❌ Input dosyası ({INPUT_FILE}) okunamadı veya işlenemedi: {e}")
        return
    
    # Sadece processed_urls'te olmayanları işlenecek listesine al
    urls_to_process = [url for url in all_urls if url not in processed_urls]
    
    if not urls_to_process:
        print("✅ Tüm ilanlar zaten işlenmiş!")
        cleanup_and_exit()
        return
    
    print(f"📊 Toplam {len(all_urls)} ilan, {len(urls_to_process)} tanesi işlenecek.")
    
    try:
        driver = init_driver()
        processed_this_session = 0
        
        for idx, url in enumerate(urls_to_process):
            # current_total_processed, tüm ilanlar içindeki gerçek sıra numarasını gösterir
            current_total_processed = all_urls.index(url) + 1 # Input listesindeki orijinal sırayı bul
            print(f"\n[{current_total_processed}/{len(all_urls)}] İşleniyor: {url}")
            
            if processed_this_session > 0 and processed_this_session % DRIVER_RESTART_INTERVAL == 0:
                print(f"\n🔄 {DRIVER_RESTART_INTERVAL} ilan işlendi, WebDriver yeniden başlatılıyor...")
                driver = init_driver()
                time.sleep(random.uniform(3, 7))
            
            ad_data = get_all_ad_data(url)
            
            # Burada kritik bir kontrol ekliyoruz: Sadece açıklama veya fiyat alanı boş değilse kaydet.
            # Bu, ilanın gerçekten işlendiği anlamına gelir.
            if ad_data.get("aciklama") is not None or ad_data.get("fiyat") is not None:
                results_list.append(ad_data)
                processed_urls.add(url) # Başarıyla çekilen URL'yi işlenmiş olarak işaretle
                processed_this_session += 1
            else:
                print(f"   ⚠️ İlan verisi eksik çekildi veya hiç çekilemedi, bu URL atlanıyor ve tekrar denenebilir: {url}")
                processed_urls.add(url)

            if len(results_list) >= SAVE_INTERVAL or (idx + 1) == len(urls_to_process):
                save_progress()
            
            time.sleep(random.uniform(1.5, 3.5))
    
    except KeyboardInterrupt:
        print("\n⚠️ Program kullanıcı tarafından durduruldu")
        cleanup_and_exit()
    except Exception as e:
        print(f"\n🚨 Kritik hata ana döngüde ({type(e).__name__}): {e}")
        cleanup_and_exit()
    finally:
        cleanup_and_exit()

# === [11] Program Başlatma ===
if __name__ == "__main__":
    main()

🚀 Araç İlanı Scraper Başlatılıyor...
📂 Mevcut 246 kayıt yüklendi (işlenmiş URL'ler).
📊 Toplam 2105 ilan, 1859 tanesi işlenecek.
✅ Yeni WebDriver oturumu başlatıldı. Global sayfa yükleme zaman aşımı: 4s

[106/2105] İşleniyor: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-business/crk-otomotiv-den-yeni-logo-2020-passat/30611327
⚠️ Temel elementler (örn. açıklama) 7s içinde gelmedi: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-business/crk-otomotiv-den-yeni-logo-2020-passat/30611327
   ❌ İlan için sayfa yüklenemedi veya temel elementler bulunamadı. Sonraki ilana geçiliyor: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-business/crk-otomotiv-den-yeni-logo-2020-passat/30611327
   ⚠️ İlan verisi eksik çekildi veya hiç çekilemedi, bu URL atlanıyor ve tekrar denenebilir: https://www.arabam.com/ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-business/crk-otomotiv-den-yeni-l